<a href="https://colab.research.google.com/github/jppego/fced_msp_g01/blob/main/SAAD_ALP_static_multi_runaway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://people.brunel.ac.uk/~mastjjb/jeb/orlib/airlandinfo.html


#Aircraft landing

There are currently 13 data files.

These data files are the test problems used in the papers "*Scheduling aircraft landings - the static case*" by J.E. Beasley, M. Krishnamoorthy, Y.M. Sharaiha and D. Abramson, Transportation  Science, vol.34, 2000, pp180-197; and

"*Displacement problem and dynamically scheduling aircraft landings*" by J.E. Beasley, M. Krishnamoorthy, Y.M. Sharaiha and
D. Abramson, Journal of the Operational Research Society, vol.55, 2004, pp54-64.

The test problems are the files:
* airland1, airland2, ..., airland13

The first eight files are those used in the Transportation Science paper referred to above.

The format of these data files is:
* number of planes (p), freeze time

for each plane i (i=1,...,p):

* appearance time, 
* earliest landing time,
* target landing time,
* latest landing time, 
* penalty cost per unit of time for landing before target,
* penalty cost per unit of time for landing after target
   
for each plane j (j=1,...p): 
* separation time required after i lands before j can land

The value of the optimal solution for each of these data
files for a varying number of runways is given in the
above papers.

The largest file is airland13 of size 800Kb (approximately). 
The entire set of files is of size 1.3Mb (approximately).

In [1]:
!pip install cplex
!pip install docplex

In [2]:
import sys
try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')      



In [3]:
''' Scheduling Aircraft Landing (Static Case) with multiple runways
    Landing times of aircrafts are determined satisfying certain constraints
    Constraints- The aircraft should land within a predetermined time interval
                 Clearance time between two landings should be satisfied if they are landing on same runway
'''
import os
import urllib.request
import numpy as np


## Retrieves datasets

In [4]:
for i in range(1,13):
  path = 'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/'
  filename =  'airland' + str(i) + '.txt'
  urllib.request.urlretrieve(path + filename, filename)
  os.listdir()


Function to read the dataset

In [5]:
#=====================================================================
# Function to fetch data from provided txt file
# @parameters- file_name : Data file(must be present in same directory)
#=====================================================================
def fetch_data(file_name):
    data=open(file_name,'r')
    lines=data.readlines()
    num_planes=int(lines[0].split()[0])
    freeze_time=int(lines[0].split()[1])

    flight_details=np.empty([num_planes,6],dtype=float)
    sep_time=np.empty([num_planes,num_planes],dtype=int)
    s=''
    for line in lines[1:]:
        s=s+line
    s=s.split()
    flag=0
    count=0
    for items in [s[x:x+6+num_planes] for x in range(0,len(s),num_planes+6)]:
        flight_details[count]=[float(x) for x in items[:6]]
        sep_time[count]=[int(x) for x in items[6:]]
        count=count+1
    print(flight_details)
    print(sep_time)
    data.close()
    return num_planes,flight_details,sep_time

# MIP model

Definitions:

* $P =$  the number of planes
* $E_i =$  the earliest landing time for plane $i$ ($i = 1, \ldots, P$)
* $L_i =$  the latest landing time for plane $i$ ($i = 1, \ldots, P$)
* $T_i =$   the target (preferred) landing time for plane $i$ ($i = 1, \ldots, P$)
* $S_{ij} =$  the required separation time ($\geq 0 $) between plane $i$ landing and plane $j$ landing (where plane $i$ lands before plane $j$), $i, j = 1, \ldots, P; i \neq j$
* $g_i =$  the penalty cost ($\geq 0 $) per unit of time for landing before the target time $T_i =$ for plane $i$ ($i = 1, \ldots, P$)
* $h_i =$  the penalty cost ($\geq 0 $) per unit of time for landing after the target time
* $T_i =$  for plane $i$ ($i = 1, \ldots, P$)
* $R =$  the number of runways 
* $ \delta_{ij} =$ 1 if plane $i$ lands before plane $j$, 0 otherwise ($i,j = 1, \ldots, P; i \neq j$)
* $z_{ij} =$  1  if planes $i$ and $j$ land on the same runaway, 0  otherwise ($i,j = 1, \ldots, P; i \neq j$)
* $y_{ij} =$  1  if plane $i$ ($i = 1, \ldots, P$) lands on runaway $r$, 0  otherwise  ($r = 1, \ldots, R$)

Decision variables:

* $x_i =$ the landing time for plane $i$ ($i = 1, \ldots, P$)
* $\alpha_i =$ how soon plane $i$ ($i = 1, \ldots, P$) lands before $T_i$
* $\beta_i =$ how soon plane $i$ ($i = 1, \ldots, P$) lands after $T_i$

Objective function:
* $ \mathrm{minimize}\sum_{i=1}^P \left( g_i \alpha_i + h_i \beta_i \right) $

Subject to:

* $ E_i \leq x_i \leq L_i, i = 1, \ldots, P $


* $ \mathrm{\textbf{W}} = [(i, j) | L_i < E_j \mathrm{~and~} L_i + S_{ij} \leq E_j, i = 1, \ldots, P; j = 1, \ldots, P; i \neq j ] $

* $ \delta_{ij} = 1 ~\forall~ \mathrm{\textbf{W}} ~ \cup \mathrm{\textbf{V}}$

* $ \alpha_i \geq T_i - x_i, i = 1, \ldots, P $

* $ 0 \leq \alpha_i \leq  T_i - E_i, i = 1, \ldots, P $

* $ \beta_i \geq x_i - T_i, i = 1, \ldots, P $

* $ 0 \leq \beta_i \leq  T_i - L_i, i = 1, \ldots, P $

* $ x_i = T_i - \alpha_i + \beta_i, i = 1, \ldots, P $

* $ \sum_{r=1}^{R} y_{ir} = 1, i = 1, \ldots, P $


* $ z_{ij}=z_{ji}, i,j = 1, \ldots, P, j > i $


* $ z_{ij} \geq y_{ir} + y_{jr} -1 , i,j = 1, \ldots, P, j > i; r, \ldots, R $

* $ x_j \geq x_i + S_{ij} z_{ij} + s_{ij} (1 - z_{ij}) - (L_i + \max(S_{ij}, s_{ij}) - E_j) \delta_{ij}, \forall (i,j)  \in  \mathrm{\textbf{U}} $

* $ \delta_{ij} \geq \frac{x_j - x_i}{L_j - E_i}, \forall (i,j) \in  \mathrm{\textbf{U}} $

* $ \sum_{i=1}^{P} \sum_{j=1, j \neq i}^{P} \delta_{ij} = P (P-1)/2 $

* $ \delta_{ij} \geq 1 - \frac{\beta_i  + \alpha_j}{T_j - T_i}, \forall (i,j) \in  \mathrm{\textbf{U}~with~}  T_i < T_j $

* $ (\alpha_i + \beta_i) + (\alpha_j + \beta_j) \geq [S_{ij} - (T_j - T_i)] \delta_{ij} + [(T_j - T_i) + S_{ji}] \delta_{ji} -\max\{[S_{ij} - (T_j - T_i)], [(T_j - T_i) + S_{ji}]\}(1 - z_{ij}),  \forall (i,j) \in  \mathrm{\textbf{U}~with~}  T_i < T_j \mathrm{~and~} (T_j - T_i) < S_{ij} $


* $ \delta_{ji} + z_{ij} \leq 1, \forall (i,j) \in  \mathrm{\textbf{U}^*} $

* $ \delta_{ji} + (1 - z_{ij})  \leq 1, \forall (i,j) \in  \mathrm{\textbf{U}^{**}} $


#Creates the MIP model

In [6]:
def indices_array_generic(m,n):
    r0 = np.arange(m) # Or r0,r1 = np.ogrid[:m,:n], out[:,:,0] = r0
    r1 = np.arange(n)
    out = np.empty((m,n,2),dtype=int)
    out[:,:,0] = r0[:,None]
    out[:,:,1] = r1
    return out

In [26]:
#=====================================================================
# Function to find landing time of the aircrafts
# @parameters- file_name Data file(must be present in same directory)
#              Number of runways
#=====================================================================
def static_ALP(fname,R):
    from docplex.mp.model import Model
    mdl = Model("Aircraft Landing Problem - Static Case - Multi Runaway")
    
    P, flight_data, sep_time=fetch_data(fname)
    try:
        #Creating a CPLEX model
        model=Model("Aircraft Landing Schedule")
       
        M = max(flight_data[:,3])-min(flight_data[:,1])

        E = flight_data[:,1]  # earliest landing time,
        T = flight_data[:,2]  # target landing time,
        L = flight_data[:,3]  # latest landing time,
        g = flight_data[:,4]  # penalty cost per unit of time for landing before target,
        h = flight_data[:,5]  # penalty cost per unit of time for landing after target

       
        g_dict = {}                          # the penalty cost per unit of time for landing before the target time
        h_dict = {}                          # the penalty cost per unit of time for landing after the target time
        for i in np.arange(P):
            g_dict[i]=flight_data[i-1,4]
            h_dict[i]=flight_data[i-1,5]

        x_dict={}                          # initializes the the penalty cost per unit of time for landing before the target time
        for i in np.arange(P):
            x_dict[i]=0

        delta_dict ={}
        z_dict ={}
        S_dict = {}
        ij = []
        for i in np.arange(P):
            for j in np.arange(P):
                delta_dict[i,j] = 0
                z_dict[i,j] = 0
                S_dict[i,j] = sep_time[i-1,j-1]
                ij.append((i,j))
        ir = []
        y_dict={}
        for i in np.arange(P):
            for r in np.arange(R):
                y_dict[i,r]=0        
                ir.append((i,r))
 

        #Adding decision variables

        alpha   = mdl.continuous_var_dict(np.arange(P),lb=0,ub=mdl.infinity, name="alpha")
        beta    = mdl.continuous_var_dict(np.arange(P),lb=0,ub=mdl.infinity, name="beta")
        x       = mdl.continuous_var_dict(np.arange(P),lb=0,ub=mdl.infinity, name="x")
        delta   = mdl.binary_var_dict(ij,lb=0,ub=1, name="delta")
        y       = mdl.binary_var_dict(ir,lb=0,ub=1, name="y")
        z       = mdl.binary_var_dict(ij,lb=0,ub=1, name="z")


        #Adding constraints
        mdl.add_constraints(x[i]>=E[i-1] for i in np.arange(P))                                                                       # Landing time of plane i must be later than the earliest landing time
        mdl.add_constraints(x[i]<=L[i-1] for i in np.arange(P))                                                                       # Landing time of plane i must be earlier than the before latest landing time
        mdl.add_constraints(delta[i,j]+delta[j,i]==1 for i in np.arange(P) for j in np.arange(P) if j!=i)                        # Either plane i lands before plane j or plane j lands before plane i, but not both
        mdl.add_constraints(alpha[i]>=T[i-1]-x[i] for i in np.arange(P))                                                              # How soon plane i lands before T[i] must be larger than T[i] - x[i]
        mdl.add_constraints(beta[i]>=x[i]-T[i-1] for i in np.arange(P))                                                               # How soon plane i lands after T[i] must be larger than x[i] - T[i]
        mdl.add_constraints(x[j]-x[i]>=S_dict[i,j]*z[j,i] - (delta[j,i])*M for i in np.arange(P) for j in np.arange(P) if j!=i)   # Separation time between plane i and plane j must be respected
        mdl.add_constraints(z[i,j]==z[j,i] for i in np.arange(P) for j in np.arange(P) if j>i)                                   # If plane i lands in the same runaway as plane j, plane j lands in the same runaway as plane i
        mdl.add_constraints(mdl.sum(y[i,r] for r in np.arange(R))==1 for i in np.arange(P))                                       # Plane i can only land in 1 runaway
        mdl.add_constraints(z[i,j]>=y[i,r]+y[j,r]-1 for r in np.arange(R) for j in np.arange(P) for i in np.arange(P) if j>i) # If there is any runaway r for which y[i,r]=y[j,r]=1 then z[i,j]=1. If z[i,j]=0 then the planes i and j cannot land on the same runaway 


        total_cost = mdl.sum(beta[i] * h[i-1] + alpha[i] * g[i-1] for i in np.arange(P))

        mdl.minimize(total_cost)
        
        mdl.print_information()

        msol = mdl.solve()
        assert msol is not None, "model can't solve"

        
                
    except mdl.error as e:
        print('Error code ' + str(e.errno) + ": " + str(e))

    except AttributeError as a:
        print('Encountered an attribute error '+str(a))

    return mdl

In [28]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [29]:
filename = 'airland1.txt'
R = 2

mdl_ = static_ALP(filename, R)
mdl_.report()
mdl_.print_solution()
        

[[ 54. 129. 155. 559.  10.  10.]
 [120. 195. 258. 744.  10.  10.]
 [ 14.  89.  98. 510.  30.  30.]
 [ 21.  96. 106. 521.  30.  30.]
 [ 35. 110. 123. 555.  30.  30.]
 [ 45. 120. 135. 576.  30.  30.]
 [ 49. 124. 138. 577.  30.  30.]
 [ 51. 126. 140. 573.  30.  30.]
 [ 60. 135. 150. 591.  30.  30.]
 [ 85. 160. 180. 657.  30.  30.]]
[[99999     3    15    15    15    15    15    15    15    15]
 [    3 99999    15    15    15    15    15    15    15    15]
 [   15    15 99999     8     8     8     8     8     8     8]
 [   15    15     8 99999     8     8     8     8     8     8]
 [   15    15     8     8 99999     8     8     8     8     8]
 [   15    15     8     8     8 99999     8     8     8     8]
 [   15    15     8     8     8     8 99999     8     8     8]
 [   15    15     8     8     8     8     8 99999     8     8]
 [   15    15     8     8     8     8     8     8 99999     8]
 [   15    15     8     8     8     8     8     8     8 99999]]
Model: Aircraft Landing Problem - Stat